In [1]:
import ee 
ee.Initialize()
import pandas as pd
import numpy as np


In [112]:
fake_geometry = ee.Geometry.Polygon([
    [[
        14.642002058678267,
        -1.5396013709395364
      ],
      [
        21.497470808678266,
        -1.5396013709395364
      ],
      [
        21.497470808678266,
        2.326761024980699
      ],
      [
        14.642002058678267,
        2.326761024980699
      ],
      [
        14.642002058678267,
        -1.5396013709395364
      ]
    ]
  ])
def zip2Image(element):
  
    value = ee.List(element).get(0)
    year = ee.List(element).get(1)
    
    return (ee.Image.constant(ee.Number.parse(value))
      .rename('ndvi')
      .clip(fake_geometry)
      .set('system:time_start', ee.Date.fromYMD(year, 1, 1).millis())
      .toFloat()
           )

def extract_landtrendr(args_list):
    
    years_list, ts_list, point_id = args_list
    ts = ee.List([str(value) for value in ts_list])
    dates = ee.List([int(year) for year in years_list])

    ts = ts.zip(dates)
    startYear = np.min(years_list)
    endYear = np.max(years_list)

    tsee = ee.ImageCollection(
      ts.map(zip2Image)
    )

    runParams = { 
        'maxSegments':            6,
        'spikeThreshold':         0.9,
        'vertexCountOvershoot':   3,
        'preventOneYearRecovery': True,
        'recoveryThreshold':      0.25,
        'pvalThreshold':          0.05,
        'bestModelProportion':    0.75,
        'minObservationsNeeded':  3,
        'timeSeries':            tsee
    }

    lt = ee.Algorithms.TemporalSegmentation.LandTrendr(**runParams).select(["LandTrendr"])

    vertexMask = lt.arraySlice(0, 3, 4); # slice out the 'Is Vertex' row - yes(1)/no(0)
    vertices = lt.arrayMask(vertexMask); # use the 'Is Vertex' row as a mask for all rows

    left = vertices.arraySlice(1, 0, -1);    # slice out the vertices as the start of segments
    right = vertices.arraySlice(1, 1, None); # slice out the vertices as the end of segments
    startYear = left.arraySlice(0, 0, 1);    # get year dimension of LT data from the segment start vertices
    startVal = left.arraySlice(0, 2, 3);     # get spectral index dimension of LT data from the segment start vertices
    endYear = right.arraySlice(0, 0, 1);     # get year dimension of LT data from the segment end vertices 
    endVal = right.arraySlice(0, 2, 3);      # get spectral index dimension of LT data from the segment end vertices

    dur = endYear.subtract(startYear);       # subtract the segment start year from the segment end year to calculate the duration of segments 
    mag = endVal.subtract(startVal);         # substract the segment start index value from the segment end index value to calculate the delta of segments
    rate = mag.divide(dur);                  # calculate the rate of spectral change

    segInfo = (
        ee.Image.cat([startYear.add(1), endYear, startVal, endVal, mag, dur, rate])
            .toArray(0)
            .mask(vertexMask.mask())
    )

    distDir = -1;

    sortByThis = segInfo.arraySlice(0,4,5).toArray(0).multiply(-1); # need to flip the delta here, since arraySort is working by ascending order
    segInfoSorted = segInfo.arraySort(sortByThis); # sort the array by magnitude
    bigDelta = segInfoSorted.arraySlice(1, 0, 1); # get the first segment in the sorted array (greatest magnitude vegetation loss segment)

    bigDeltaImg = ee.Image.cat(bigDelta.arraySlice(0,0,1).arrayProject([1]).arrayFlatten([['yod']]),
    bigDelta.arraySlice(0,1,2).arrayProject([1]).arrayFlatten([['endYr']]),
    bigDelta.arraySlice(0,2,3).arrayProject([1]).arrayFlatten([['startVal']]).multiply(distDir),
    bigDelta.arraySlice(0,3,4).arrayProject([1]).arrayFlatten([['endVal']]).multiply(distDir),
    bigDelta.arraySlice(0,4,5).arrayProject([1]).arrayFlatten([['mag']]).multiply(distDir),
    bigDelta.arraySlice(0,5,6).arrayProject([1]).arrayFlatten([['dur']]),
    bigDelta.arraySlice(0,6,7).arrayProject([1]).arrayFlatten([['rate']]).multiply(distDir));

    distMask =  bigDeltaImg.select(['mag']).lt(1000).And(bigDeltaImg.select(['dur']).lt(5));

    bigFastDist = bigDeltaImg  #.mask(distMask).int16(); // need to set as int16 bit to use connectedPixelCount for minimum mapping unit filter

    landtrendr = bigFastDist.select(['mag', 'dur', 'yod', 'rate', 'endYr']).clip(fake_geometry).reduceRegion(**{
      'reducer': ee.Reducer.first(),
      'scale': 3000
    }).getInfo()

    return landtrendr['mag'], landtrendr['dur'], landtrendr['yod'], landtrendr['rate'],  landtrendr['endYr'], point_id

In [113]:
df = pd.read_pickle('/home/vollrath/Ethiopia/ts_analysis_oldSus/sbae_point_analysis/results_ref/results_Landsat_ndfi_2010-01-01_2013-01-01_2018-01-01_0.25.pickle')

In [114]:
dates = df.head(1).dates.values[0]
ts = np.array(df.head(1).ts.values[0])
len(dates), len(ts)

(105, 105)

In [115]:
args_list = []
for i, row in df.iterrows():

    years = np.unique([date.year for date in row.dates])
    ts_yearly = []

    for year in years:
        
        idx = np.array([True if date.year == year else False for date in row.dates])
        ts_yearly.append(np.nanmean(np.array(row.ts)[idx]))
        
    args_list.append([years, ts_yearly, row.point_id])
    if i > 3:
        break
#extract_landtrendr(years, ts_yearly)

In [116]:
from godale import Executor

In [117]:

d ={}
executor = Executor(executor="concurrent_threads", max_workers=16)
for i, task in enumerate(executor.as_completed(
    func=extract_landtrendr,
    iterable=args_list
)):
    try:
        d[i] = list(task.result())
    except ValueError:
        print("timescan task failed")

landtrendr_df = pd.DataFrame.from_dict(d, orient='index')
landtrendr_df.columns = ['ltr_magnitude', 'ltr_dur', 'ltr_yod', 'ltr_rate', 'ltr_end_year', 'point_id']

In [118]:
landtrendr_df

,ltr_magnitude,ltr_dur,ltr_yod,ltr_rate,ltr_end_year,point_id
0,-151.452474,7,2010,-21.636068,2016,32932
1,0.000000,7,2010,0.000000,2016,32941
2,-264.949148,7,2010,-37.849878,2016,32933
3,-165.923646,7,2010,-23.703378,2016,32931
4,-174.507507,2,2010,-87.253753,2011,32942


In [1]:
d = {'key': 2}

In [2]:
d.update(timeSeries=23)

In [4]:
d.pop('timeSeries', None)

23

In [125]:
del d['timeSeries']

In [5]:
d

{'key': 2}